In [8]:
import os
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

tqdm.pandas()

In [ ]:
csv_path = '../data/fixed/formatted_tasks.csv'
df = pd.read_csv(csv_path, encoding='utf-8')

print(f"=== CARREGANDO TAREFAS FORMATADAS ===")
print(f"Shape: {df.shape}")
print(f"Colunas: {df.columns.tolist()}")
print(f"Primeiras 5 linhas:")
print(df.head())

if 'TASK_EN' not in df.columns:
    df['TASK_EN'] = None
    print("Coluna TASK_EN adicionada")

print(f"\nTarefas ainda não traduzidas: {df['TASK_EN'].isnull().sum()}")
print(f"Total de tarefas: {len(df)}")

calendar_unique_list = df['CALENDAR'].unique().tolist()
print(f"Calendários únicos: {calendar_unique_list}")

=== CARREGANDO TAREFAS FORMATADAS ===
Shape: (320, 4)
Colunas: ['SUMMARY', 'CALENDAR', 'TASK', 'TASK_EN']
Primeiras 5 linhas:
           SUMMARY     CALENDAR                 TASK                  TASK_EN
0    cafe da manha  Alimentação  Tomar café da manhã           Have breakfast
1  lanche da manha  Alimentação     Lanchar de manhã     Have a morning snack
2           almoco  Alimentação              Almoçar               Have lunch
3  lanche da tarde  Alimentação      Lanchar à tarde  Have an afternoon snack
4       pre-treino  Alimentação   Tomar o pré-treino         Take pre-workout

Tarefas ainda não traduzidas: 9
Total de tarefas: 320
Calendários únicos: ['Alimentação', 'PUC', 'Pessoal', 'Saúde', 'Compromisso', 'Exames', 'Social', 'Games', 'Exercícios', 'Trabalho', 'Viagens']


In [ ]:
client = OpenAI(
    api_key=os.getenv('GEMINI_API_KEY'),
    base_url=os.getenv('GEMINI_BASE_URL')
)

def translate_task(row):
    """
    Traduz uma tarefa do português para o inglês.
    """
    try:
        response = client.chat.completions.create(
            model="gemini-2.5-flash",
            temperature=0.2,
            max_tokens=1024,
            messages=[
                {
                    "role": "system",
                    "content": """Você é um tradutor especializado em traduzir tarefas do português para o inglês.

Suas regras:
1. Traduza a tarefa de forma natural e idiomática para o inglês
2. Mantenha o sentido original da ação
3. Use verbos apropriados em inglês (ex: "Tomar café da manhã" → "Have breakfast")
4. Para contextos acadêmicos, use terminologia adequada
5. Para atividades específicas do Brasil, adapte quando necessário
6. Quando a tarefa tiver muitos substativos próprios ou nomes que não precisam ser traduzidos, traduza apenas o verbo ou a ação principal e mantenha os nomes ou eventos específicos em português
7. Retorne apenas a tradução, sem explicações adicionais


Exemplos:
- "Tomar café da manhã" → "Have breakfast"
- "Ir para consulta com o psiquiatra" → "Go to psychiatrist appointment"
- "Fazer a prova 1 de Cálculo" → "Take Calculus exam 1"
- "Aula da disciplina INF1410" → "Attend INF1410 class"
"""
                },
                {
                    "role": "user",
                    "content": f"Traduza esta tarefa para o inglês: \"{row['TASK']}\""
                }
            ],
        )

        row['TASK_EN'] = response.choices[0].message.content.strip()
        return row
    except Exception as e:
        print(f"Erro ao traduzir '{row['TASK']}': {e}")
        row['TASK_EN'] = None
        return row

In [ ]:
print("=== INICIANDO TRADUÇÃO DAS TAREFAS ===")

df_to_translate = df[df['TASK_EN'].isnull()].copy()

if len(df_to_translate) > 0:
    print(f"Traduzindo {len(df_to_translate)} tarefas...")

    df_translated = df_to_translate.progress_apply(translate_task, axis=1)

    df.update(df_translated)

    print("Tradução concluída!")
else:
    print("Todas as tarefas já foram traduzidas!")

print(f"\nTarefas traduzidas: {df['TASK_EN'].notnull().sum()}")
print(f"Tarefas pendentes: {df['TASK_EN'].isnull().sum()}")

=== INICIANDO TRADUÇÃO DAS TAREFAS ===
Traduzindo 9 tarefas...


100%|██████████| 9/9 [00:34<00:00,  3.88s/it]

Erro ao traduzir 'ia &': 'NoneType' object has no attribute 'strip'
✅ Tradução concluída!

Tarefas traduzidas: 319
Tarefas pendentes: 1


In [ ]:
print("=== EXEMPLOS DE TRADUÇÕES ===")
examples = df[df['TASK_EN'].notnull()].head(10)

for idx, row in examples.iterrows():
    print(f"Original: {row['TASK']}")
    print(f"Traduzida: {row['TASK_EN']}")
    print(f"Calendário: {row['CALENDAR']}")
    print("-" * 50)

=== EXEMPLOS DE TRADUÇÕES ===
Original: Tomar café da manhã
Traduzida: Have breakfast
Calendário: Alimentação
--------------------------------------------------
Original: Lanchar de manhã
Traduzida: Have a morning snack
Calendário: Alimentação
--------------------------------------------------
Original: Almoçar
Traduzida: Have lunch
Calendário: Alimentação
--------------------------------------------------
Original: Lanchar à tarde
Traduzida: Have an afternoon snack
Calendário: Alimentação
--------------------------------------------------
Original: Tomar o pré-treino
Traduzida: Take pre-workout
Calendário: Alimentação
--------------------------------------------------
Original: Jantar
Traduzida: Have dinner
Calendário: Alimentação
--------------------------------------------------
Original: Aula da disciplina INF1410 - Gerência de Projetos de Informática
Traduzida: Attend INF1410 - IT Project Management class
Calendário: PUC
--------------------------------------------------
Original:

# Exportar DataFrame com Traduções

Salvar o DataFrame com as traduções em um novo arquivo CSV.

In [ ]:
# Filtrar apenas tarefas que possuem tradução e não têm "!" no SUMMARY
print("=== FILTRANDO TAREFAS ===")
df_original_size = len(df)

# Primeiro filtro: apenas tarefas com tradução
df_with_translation = df[df['TASK_EN'].notnull()].copy()
df_without_translation = df[df['TASK_EN'].isnull()].copy()

print(f"Tarefas com tradução: {len(df_with_translation)}")
print(f"Tarefas sem tradução (removidas): {len(df_without_translation)}")

if len(df_without_translation) > 0:
    print("\n=== TAREFAS REMOVIDAS - SEM TRADUÇÃO ===")
    for idx, row in df_without_translation.iterrows():
        print(f"- {row['TASK']} (Calendar: {row['CALENDAR']})")

# Segundo filtro: remover tarefas com "!" no SUMMARY
df_filtered = df_with_translation[~df_with_translation['SUMMARY'].str.contains('!', na=False)].copy()
df_with_exclamation = df_with_translation[df_with_translation['SUMMARY'].str.contains('!', na=False)].copy()

print(f"\nTarefas após filtro de tradução: {len(df_with_translation)}")
print(f"Tarefas após filtro de '!' no SUMMARY: {len(df_filtered)}")
print(f"Tarefas com '!' no SUMMARY (removidas): {len(df_with_exclamation)}")

if len(df_with_exclamation) > 0:
    print("\n=== TAREFAS REMOVIDAS - COM '!' NO SUMMARY ===")
    for idx, row in df_with_exclamation.iterrows():
        print(f"- {row['TASK']} (Summary: {row['SUMMARY']}) (Calendar: {row['CALENDAR']})")

print(f"\nTarefas originais: {df_original_size}")
print(f"Total de tarefas removidas: {df_original_size - len(df_filtered)}")

# Salvar o DataFrame filtrado com as traduções
output_path = '../data/fixed/translated_tasks.csv'
df_filtered.to_csv(output_path, index=False, encoding='utf-8')

print(f"\nArquivo salvo em: {output_path}")
print(f"Total de linhas: {len(df_filtered)}")
print(f"Colunas: {df_filtered.columns.tolist()}")

# Verificar algumas estatísticas finais
print(f"\n=== ESTATÍSTICAS FINAIS ===")
print(f"Dataset final: {len(df_filtered)} tarefas limpas e traduzidas")
print(f"Filtros aplicados: tradução completa + sem '!' no SUMMARY")

=== FILTRANDO TAREFAS ===
Tarefas com tradução: 319
Tarefas sem tradução (removidas): 1

=== TAREFAS REMOVIDAS - SEM TRADUÇÃO ===
- ia & (Calendar: Pessoal)

Tarefas após filtro de tradução: 319
Tarefas após filtro de '!' no SUMMARY: 290
Tarefas com '!' no SUMMARY (removidas): 29

=== TAREFAS REMOVIDAS - COM '!' NO SUMMARY ===
- Dar parabéns (Summary: parabens!) (Calendar: Pessoal)
- Ir para rm (Summary: !rm) (Calendar: Compromisso)
- Ir para zion (Summary: !zion) (Calendar: Compromisso)
- Visitar a rede (Summary: !visita rede) (Calendar: Compromisso)
- mago da musica link (Summary: !mago da musica link) (Calendar: Compromisso)
- Consultar Viviane Nutri (Summary: !viviane nutri) (Calendar: Saúde)
- Agendar consulta com dra. Vania (Summary: !dra. vania) (Calendar: Saúde)
- Encontrar a academia ideal (Summary: !academia ideal) (Calendar: Exercícios)
- Fazer a aula da Liane (Summary: !aula da liane) (Calendar: Pessoal)
- Ir à casa do Felipe (Summary: !casa do felipe) (Calendar: Pessoal)
-